In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydrake.all
import os
from IPython.display import display

from pydrake.all import RigidTransform, RotationMatrix, RollPitchYaw, RevoluteSpring, LogOutput, SignalLogger, ForceElement, DoorHinge, DoorHingeConfig, PidController, FixedInputPortValue
import pydrake.multibody.jupyter_widgets
import pydrake.systems.jupyter_widgets
from ipywidgets import FloatSlider, Layout

import matplotlib.pyplot as plt

In [ ]:
# Constants
in2m = 0.0254

PAPER_WIDTH = 11*in2m
PAPER_DEPTH = 8.5*in2m
PAPER_HEIGHT = 0.005
PAPER_DENSITY = 80/1000

PEDESTAL_WIDTH = 0.225
PEDESTAL_HEIGHT = 0.1

epsilon = 0.001

In [ ]:
def AddPedestal(plant):
    mu = 5.0
    # Parse pedestal model
    parser = pydrake.multibody.parsing.Parser(plant)
    pedestal_instance = parser.AddModelFromFile("pedestal.sdf")
    
    pedestal_body = plant.GetBodyByName("pedestal_base", pedestal_instance)
    width=PEDESTAL_WIDTH
    depth=0.2
    height=0.1
    i=0
    for x in [-width/2.0, width/2.0]:
        for y in [-depth/2.0, depth/2.0]:
            for z in [-height/2.0, height/2.0]:
                plant.RegisterCollisionGeometry(
                    pedestal_body,
                    RigidTransform([x, y, z]),
                    pydrake.geometry.Sphere(radius=1e-7), f"_contact_sphere{i}",
                    pydrake.multibody.plant.CoulombFriction(mu, mu))
                i += 1
    
    # Weld pedestal to world
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("pedestal_base", pedestal_instance),
        RigidTransform(RotationMatrix(), [0, 0, 0.1/2])
    )
    
    return pedestal_instance

In [ ]:
def AddPaper(plant, mass=None, mu=5.0, width=PAPER_WIDTH, depth=PAPER_DEPTH, height=PAPER_HEIGHT, name="paper"):
    if mass is None:
        mass = PAPER_DENSITY*width*height
    
    paper_instance = plant.AddModelInstance(name)
    paper_body = plant.AddRigidBody(
        name + "_body",
        paper_instance,
        pydrake.multibody.tree.SpatialInertia(mass=mass, p_PScm_E=np.array([0., 0., 0.]),
        G_SP_E=pydrake.multibody.tree.UnitInertia.SolidBox(width, depth, height))
    )
    
    # Set up collision
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(
            paper_body,
            RigidTransform(),
            pydrake.geometry.Box(width, depth, height),
            name + "_body", pydrake.multibody.plant.CoulombFriction(mu, mu)
        )
        i=0
        for x in [-width/2.0, width/2.0]:
            for y in [-depth/2.0, depth/2.0]:
                for z in [-height/2.0, height/2.0]:
                    plant.RegisterCollisionGeometry(
                        paper_body,
                        RigidTransform([x, y, z]),
                        pydrake.geometry.Sphere(radius=1e-7), f"contact_sphere{i}",
                        pydrake.multibody.plant.CoulombFriction(mu, mu))
                    i += 1
        plant.RegisterVisualGeometry(
            paper_body,
            RigidTransform(),
            pydrake.geometry.Box(width, depth, height),
            name + "_body",
            [0.9, 0.9, 0.9, 1.0])
    
    return paper_instance, paper_body

In [ ]:
def AddPointFinger(plant):
    mu = 1.0
    finger = plant.AddModelInstance("finger")
    false_body1 = plant.AddRigidBody(
        "false_body1",
        finger,
        pydrake.multibody.tree.SpatialInertia(0, [0,0,0], pydrake.multibody.tree.UnitInertia(0,0,0)))
    finger_body = plant.AddRigidBody(
        "body", 
        finger, 
        pydrake.multibody.tree.SpatialInertia(mass=1.0, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=pydrake.multibody.tree.UnitInertia(1.0, 1.0, 1.0)))
    shape = pydrake.geometry.Sphere(0.01)
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(
            finger_body, RigidTransform(), shape, "body", pydrake.multibody.plant.CoulombFriction(mu, mu))
        plant.RegisterVisualGeometry(finger_body, RigidTransform(), shape, "body", [.9, .5, .5, 1.0])
    finger_x = plant.AddJoint(
        pydrake.multibody.tree.PrismaticJoint(
            "finger_x",
            plant.world_frame(),
            plant.GetFrameByName("false_body1"), [1, 0, 0], -.3, .3))
    plant.AddJointActuator("finger_x", finger_x)
    finger_z = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint(
        "finger_z", plant.GetFrameByName("false_body1"), plant.GetFrameByName("body"), [0, 0, 1], 0.0, 0.5))
    finger_z.set_default_translation(0.25)
    plant.AddJointActuator("finger_z", finger_z)

    return finger

class PointFingerForceControl(pydrake.systems.framework.LeafSystem):

    def __init__(self, plant):
        pydrake.systems.framework.LeafSystem.__init__(self)
        self._plant = plant

        self.DeclareVectorInputPort(
            "desired_contact_force", pydrake.systems.framework.BasicVector(2))
        self.DeclareVectorOutputPort(
            "finger_actuation", pydrake.systems.framework.BasicVector(2), 
                                    self.CalcOutput)

    def CalcOutput(self, context, output):
        finger_mass = 1
        g = self._plant.gravity_field().gravity_vector()[[0,2]]

        desired_force = self.get_input_port(0).Eval(context)
        output.SetFromVector(-finger_mass*g - desired_force)

In [ ]:
class TorsionalDamper(ForceElement):
    def __init__(self):
        super().__init__()
        self.damping_constant = 1 #damping_constant
        
    def CalcAndAddForceContribution(context, pc, vc):
        pass
    
    def CalcPotentialEnergy(context, pc):
        pass
    
    def CalcConservativePower(context, pc, vc):
        pass
    
    def CalcNonConservativePower(context, pc, vc):
        pass

In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()

# Initialize multibody plant with all objects
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=1e-2)
pedestal_instance = AddPedestal(plant)
paper1_instance, paper1_body = AddPaper(plant, width=PAPER_WIDTH/2, name="paper1")
paper2_instance, paper2_body = AddPaper(plant, width=PAPER_WIDTH/2, name="paper2")

# Paper to paper hinge
paper1_hinge_frame = pydrake.multibody.tree.FixedOffsetFrame(
    "paper1_hinge_frame",
    plant.GetBodyByName("paper1_body", paper1_instance),
    RigidTransform(RotationMatrix(), [PAPER_WIDTH/4+epsilon/2, 0, 0.5*PAPER_HEIGHT]))
plant.AddFrame(paper1_hinge_frame)
paper2_hinge_frame = pydrake.multibody.tree.FixedOffsetFrame(
    "paper2_hinge_frame",
    plant.GetBodyByName("paper2_body", paper2_instance),
    RigidTransform(RotationMatrix(), [(-PAPER_WIDTH/4+epsilon/2), 0, 0.5*PAPER_HEIGHT]))
plant.AddFrame(paper2_hinge_frame)

joint = plant.AddJoint(pydrake.multibody.tree.RevoluteJoint(
    "paper_hinge",
    paper1_hinge_frame,
    paper2_hinge_frame,
    [0, 1, 0]))

joint.set_default_angle(-np.pi/12)

plant.WeldFrames(
    plant.world_frame(),
    plant.GetBodyByName("paper1_body", paper1_instance).body_frame(),
    RigidTransform(RotationMatrix(), [-(PEDESTAL_WIDTH/2-PAPER_WIDTH/4), 0, 0.1+PEDESTAL_HEIGHT+PAPER_HEIGHT/2])
)

# DoorHingeConfig.catch_torque = 0
# DoorHingeConfig.catch_width = 0
# DoorHingeConfig.dynamic_friction_torque = 0
# DoorHingeConfig.motion_threshold = 0
# DoorHingeConfig.spring_constant = 0.00005
# DoorHingeConfig.spring_zero_angle_rad = -np.pi/2
# DoorHingeConfig.static_friction_torque = 0
# DoorHingeConfig.viscous_friction = 0
# dhc = DoorHingeConfig()
# plant.AddForceElement(DoorHinge(joint=joint, config=dhc))
# plant.AddForceElement(RevoluteSpring(joint=joint, nominal_angle=-np.pi/2, stiffness=0.00005))


ja = plant.AddJointActuator("joint actuator", joint)

ctrlr = PidController(kp=[[0.0001]], ki=[[0]], kd=[[0.00001]])
builder.AddSystem(ctrlr)

In [ ]:
plant.Finalize()

builder.Connect(ctrlr.get_output_port(), plant.get_actuation_input_port(paper2_instance))
builder.Connect(plant.get_state_output_port(paper2_instance), ctrlr.get_input_port_estimated_state())

poses_logger = LogOutput(plant.get_state_output_port(), builder)

# Set up visualizer
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)

# Set up diagram and context
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

mbp_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
c_context = diagram.GetMutableSubsystemContext(ctrlr, diagram_context)
ctrlr.get_input_port_desired_state().FixValue(c_context, [-np.pi/2, 0])



In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
simulator.AdvanceTo(5)
vis.stop_recording()
vis.publish_recording()

In [ ]:
%matplotlib notebook

plt.plot(poses_logger.sample_times(), poses_logger.data()[0,:], label="Position")
plt.plot(poses_logger.sample_times(), poses_logger.data()[1,:], label="Velocity")
plt.axhline(-np.pi/2, linestyle='--', color='k', label='Setpoint')
plt.legend()
plt.xlabel("Time (s)")
plt.show()